In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import seaborn as sb

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import mean_squared_error
from sklearn import tree
import xgboost as xgb

from sklearn.model_selection import train_test_split

In [2]:

from sklearn.preprocessing import OneHotEncoder

def normaliza_df(df, onehotencoder = None):
    print(df.shape)

    # drops -------------------------------------------------
    '''
      removemos la columna de la cual el modelo podria aprender que su existencia
      implica el Closed_won (ya que es un valor que se obtiene a posteriori de haber
      ganado y estaria mal usarlo ya que los modelos entrenadas con ella serian 
      incapaces de poder predecir correctamente)
    '''
    df = df.drop(['Sales_Contract_No'], axis=1)
    
    # irrelevantes (se eliminan) ----------------------------------------------
    
    # Total_Taxable_Amount = Sum rows[Total_Amount]
    
    # preparamos la columna Total_Taxable_Amount para ser aplanada
    df["Total_Amount_sum"] = df.groupby("Opportunity_ID")["Total_Amount"].transform("sum")

    df = df.drop(['Total_Taxable_Amount'], axis=1)
    df = df.drop(['Total_Amount_Currency'], axis=1)

    
    # esta columna es equivalente a oportunity_id
    df = df.drop(['Opportunity_Name'], axis=1)
    
    # dato cte = NaT
    df = df.drop(['Last_Activity'], axis=1)
    
    # todo: convertir estas en una ventana de tiempo
    df['Planned_Delivery_Start_Date'] = pd.to_datetime(df['Planned_Delivery_Start_Date'], format="%m/%d/%Y")
    df['Planned_Delivery_End_Date'] = pd.to_datetime(df['Planned_Delivery_End_Date'], format="%m/%d/%Y")
    
    df['Planned_Delivery_Date_diff'] = (df['Planned_Delivery_End_Date'] - df['Planned_Delivery_Start_Date']).dt.days
    
    df = df.drop(['Planned_Delivery_Start_Date'], axis=1)
    df = df.drop(['Planned_Delivery_End_Date'], axis=1)
    
    # @todo: no estoy seguro si esto aporta
    df = df.drop(['Quote_Expiry_Date'], axis=1)
    
    # @todo: convertirla a una diferencia de dias?
    df = df.drop(['Last_Modified_Date'], axis=1)
    
    # @todo: existe Opportunity_id 
    df = df.drop(['ID'], axis=1)

    # basado en lo charlado con el grupo, ahora subdividimos el df de forma de tener
    # en cuenta los casos cerrados y cobertimos las varaibles categorias en su
    # representacion mas simple (siendo dos casos en Cloased_Won = 1 y Closed:_lost = 0)
    if 'Stage' in df.columns:
        df = df[((df['Stage'] == 'Closed Won') | (df['Stage'] == 'Closed Lost'))]
        df['Stage'] = df['Stage'].apply(lambda x: 1 if x == 'Closed Won' else 0)
    
    # hacemos que las variables temporales en las que nos vanos a enfocar sean del tipo correcto
    df['Account_Created_Date'] = pd.to_datetime(df['Account_Created_Date'], format="%m/%d/%Y")
    df['Opportunity_Created_Date'] = pd.to_datetime(df['Opportunity_Created_Date'], format="%m/%d/%Y")
    
    df['Quote_Type'] = df['Quote_Type'].apply(lambda x: 1 if x == 'Binding' else 0)
    
    #------------------------------------------
    df = df.sort_values(by="Opportunity_Created_Date")
    
    df = df.drop(columns = 'Opportunity_Created_Date')
    df = df.drop(columns = 'Account_Created_Date')
    #------------------------------------------
    
    categoric_cols = df.columns[df.dtypes==object].tolist() 
    numeric_cols = df.columns[df.dtypes=='float64'].tolist() 
    numeric_cols_2 = df.columns[df.dtypes=='int64'].tolist() 
    date_cols = df.columns[df.dtypes=='datetime64[ns]'].tolist() 

    if 'Stage' in df.columns:
        onehotencoder = OneHotEncoder(handle_unknown = 'ignore')
        onehotencoder.fit(df[categoric_cols])
    
    cat_rel = 0
    all_col_names = []
    for cat in onehotencoder.categories_:
        for col in cat:
            all_col_names.append(categoric_cols[cat_rel] + '_' + col)
        cat_rel = cat_rel + 1
    
    categorical = pd.DataFrame(onehotencoder.transform(df[categoric_cols]).toarray(), columns=all_col_names)
    
    categorical = categorical.reset_index()
    df = df.reset_index()
    
    print('ante ...')
    print(df.shape)
    print(categorical.shape)
    print(df[numeric_cols].shape)
    print(df[numeric_cols_2].shape)

    frames_to_concat = [categorical, df[numeric_cols], df[numeric_cols_2]]
    df_r =  pd.concat(frames_to_concat, axis=1)
       
    df_r = df_r.drop(columns = 'index')
    
    return df_r, onehotencoder

# Binary Encoding

In [57]:
train_binary =  pd.read_csv('/home/leandro/Documentos/Organizacion de datos/Fiuba-Big-Data-Analytics-TPs/Tp 2/Datos/TrainBinaryEncoding.csv')
test_binary =  pd.read_csv('/home/leandro/Documentos/Organizacion de datos/Fiuba-Big-Data-Analytics-TPs/Tp 2/Datos/TestBinaryEncoding.csv')

test_binary = test_binary.drop(columns ="Unnamed: 0")
train_binary = train_binary.drop(columns ="Unnamed: 0")



X_train, X_test, y_train, y_test = \
    train_test_split(train_binary.drop(columns = 'Stage'), train_binary['Stage'], test_size=0.2, random_state=123)


In [58]:

model_r = RandomForestRegressor(random_state = 0)
model_r.fit(X_train, y_train)
predict_r = model_r.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predict_r))
print("RMSE: %f" % (rmse))

predict_r

RMSE: 0.268029


array([0.91, 0.04, 1.  , ..., 0.25, 0.57, 0.97])

In [59]:
model_r2 = RandomForestClassifier(random_state = 0)
model_r2.fit(X_train, y_train)
predict_r2 = model_r2.predict_proba(X_test)
rmse2 = np.sqrt(mean_squared_error(y_test, predict_r2[:,1]))
print("RMSE: %f" % (rmse2))

predict_r2

RMSE: 0.260197


array([[0.12, 0.88],
       [0.95, 0.05],
       [0.  , 1.  ],
       ...,
       [0.82, 0.18],
       [0.84, 0.16],
       [0.01, 0.99]])

In [74]:
predicts = model_r2.predict_proba(test_binary.fillna(0))
predicts

array([[0.5 , 0.5 ],
       [0.39, 0.61],
       [0.34, 0.66],
       ...,
       [0.48, 0.52],
       [0.39, 0.61],
       [0.7 , 0.3 ]])

In [77]:
output = pd.DataFrame({'Opportunity_ID':test_binary.Opportunity_ID, 'Target': predicts[:,1]})
output = output.groupby('Opportunity_ID').mean()
output

,Target
Opportunity_ID,
10689,0.590000
10690,0.518000
10691,0.660000
10692,0.593333
10693,0.780000
...,...
12364,0.610000
12365,0.580000
12366,0.520000


# One Hot

In [78]:
test = pd.read_csv('/home/leandro/Documentos/Organizacion de datos/Fiuba-Big-Data-Analytics-TPs/Tp 2/Datos/Test_TP2_Datos_2020-2C.csv')

train = pd.read_csv('/home/leandro/Documentos/Organizacion de datos/Fiuba-Big-Data-Analytics-TPs/Tp 2/Datos/Train_TP2_Datos_2020-2C.csv')

In [89]:
[train_onehot, one_hot] = normaliza_df(train)
[test_onehot, _] = normaliza_df(test, one_hot)

train_onehot =train_onehot.fillna(0)
test_onehot = test_onehot.fillna(0)

(16947, 52)
ante ...
(16883, 43)
(16883, 2853)
(16883, 5)
(16883, 10)
(2551, 51)
ante ...
(2551, 42)
(2551, 2853)
(2551, 5)
(2551, 9)


In [83]:
X_train, X_test, y_train, y_test = \
    train_test_split(train_onehot.drop(columns = 'Stage'), train_onehot['Stage'], test_size=0.2, random_state=123)



In [84]:
model_r2 = RandomForestClassifier(random_state = 0)
model_r2.fit(X_train, y_train)
predict_r2 = model_r2.predict_proba(X_test)
rmse2 = np.sqrt(mean_squared_error(y_test, predict_r2[:,1]))
print("RMSE: %f" % (rmse2))

predict_r2

RMSE: 0.249198


array([[0.15, 0.85],
       [0.01, 0.99],
       [0.16, 0.84],
       ...,
       [0.25, 0.75],
       [0.87, 0.13],
       [0.86, 0.14]])

In [90]:
predicts = model_r2.predict_proba(test_onehot)
predicts

array([[0.34, 0.66],
       [0.17, 0.83],
       [0.19, 0.81],
       ...,
       [0.3 , 0.7 ],
       [0.42, 0.58],
       [0.7 , 0.3 ]])

In [92]:
output = pd.DataFrame({'Opportunity_ID':test_onehot.Opportunity_ID, 'Target': predicts[:,1]})
output = output.groupby('Opportunity_ID').mean()
output

,Target
Opportunity_ID,
10689,0.680000
10690,0.634000
10691,0.730000
10692,0.751667
10693,0.930000
...,...
12364,0.770000
12365,0.700000
12366,0.580000


# Archivos de fernando


In [4]:
test_fer = pd.read_csv('/home/leandro/Documentos/Organizacion de datos/Datos Pesados/210227_tp2_test_feng.csv')
train_fer = pd.read_csv('/home/leandro/Documentos/Organizacion de datos/Datos Pesados/210227_tp2_train_feng.csv')

In [5]:
X_train, X_test, y_train, y_test = \
    train_test_split(train_fer.drop(columns = 'Stage'), train_fer['Stage'], test_size=0.2, random_state=123)


In [6]:
model_r2 = RandomForestClassifier(random_state = 0)
model_r2.fit(X_train, y_train)
predict_r2 = model_r2.predict_proba(X_test)
rmse2 = np.sqrt(mean_squared_error(y_test, predict_r2[:,1]))
print("RMSE: %f" % (rmse2))

predict_r2

RMSE: 0.316260


array([[0.21, 0.79],
       [0.83, 0.17],
       [0.62, 0.38],
       ...,
       [0.05, 0.95],
       [1.  , 0.  ],
       [0.67, 0.33]])

In [7]:
predicts = model_r2.predict_proba(test_fer)
predicts

array([[0.39, 0.61],
       [0.5 , 0.5 ],
       [0.26, 0.74],
       ...,
       [0.56, 0.44],
       [0.42, 0.58],
       [0.81, 0.19]])

In [9]:
output = pd.DataFrame({'Opportunity_ID':test_fer.Opportunity_ID, 'Target': predicts[:,1]})
output = output.groupby('Opportunity_ID').max()
output

,Target
Opportunity_ID,
10689,0.61
10690,0.50
10691,0.74
10692,0.57
10693,0.83
...,...
12364,0.75
12365,0.54
12366,0.44


In [11]:
output.to_csv('/home/leandro/Documentos/Organizacion de datos/Fiuba-Big-Data-Analytics-TPs/Tp 2/Resultados/RandomForestScore4.csv')